# 🚀 MOUAADNET-ULTRA Training
## Human Detection with PA-100k Dataset

**Lead Architect:** MOUAAD IDOUFKIR

This notebook trains MOUAADNET-ULTRA for human detection using the PA-100k pedestrian attribute dataset.

---

### 📋 Steps:
1. Setup environment
2. Download PA-100k dataset
3. Prepare data loaders
4. Train model
5. Export to ONNX

## 1️⃣ Environment Setup

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone MOUAADNET-ULTRA repository
!git clone https://github.com/mouaadidoufkir/mouaadnet-ultra.git
%cd mouaadnet-ultra

# Or upload your local files
# from google.colab import files
# files.upload()

In [ ]:
# Install dependencies
!pip install -q torch torchvision tqdm scipy

## 2️⃣ Download PA-100k Dataset

In [ ]:
# Option 1: Download from Google Drive (if you have it uploaded)
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/PA-100k ./data/

# Option 2: Download from source
# PA-100k: https://github.com/xh-liu/HydraPlus-Net#pa-100k-dataset
# You need to request access from the authors

# Create data directory
!mkdir -p data/PA-100k/images

print("📁 Please upload PA-100k dataset to data/PA-100k/")
print("   Expected structure:")
print("   data/PA-100k/")
print("   ├── images/")
print("   │   ├── 000001.jpg")
print("   │   ├── 000002.jpg")
print("   │   └── ...")
print("   └── annotation.mat")

In [ ]:
# Upload dataset (alternative method)
from google.colab import files

print("Upload PA-100k.zip file:")
uploaded = files.upload()

# Unzip
!unzip -q PA-100k.zip -d data/
!ls data/PA-100k/

## 3️⃣ Dataset & DataLoader

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from scipy.io import loadmat
from tqdm import tqdm

class PA100kDataset(Dataset):
    """
    PA-100k Dataset for Human Detection Training.
    
    Since PA-100k contains cropped pedestrians, we use it to:
    1. Train a person classifier (is this a person?)
    2. Extract gender attribute (Male/Female)
    
    For detection, we generate synthetic heatmaps centered on the image.
    """
    
    # PA-100k attribute indices
    # Gender is typically attributes 0-1: Female, Male
    GENDER_FEMALE_IDX = 0
    GENDER_MALE_IDX = 1
    
    def __init__(self, root_dir, split='train', img_size=416, transform=None):
        self.root_dir = root_dir
        self.img_size = img_size
        self.transform = transform or self._default_transform()
        
        # Load annotations
        anno_path = os.path.join(root_dir, 'annotation.mat')
        if os.path.exists(anno_path):
            anno = loadmat(anno_path)
            
            # Get file names and labels
            if split == 'train':
                self.images = [str(x[0][0]) for x in anno['train_images_name']]
                self.labels = anno['train_label']
            elif split == 'val':
                self.images = [str(x[0][0]) for x in anno['val_images_name']]
                self.labels = anno['val_label']
            else:  # test
                self.images = [str(x[0][0]) for x in anno['test_images_name']]
                self.labels = anno['test_label']
        else:
            # Fallback: load all images from directory
            img_dir = os.path.join(root_dir, 'images')
            self.images = sorted(os.listdir(img_dir))
            self.labels = None
        
        print(f"Loaded {len(self.images)} images for {split} split")
    
    def _default_transform(self):
        return transforms.Compose([
            transforms.Resize((self.img_size, self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])
    
    def __len__(self):
        return len(self.images)
    
    def _generate_heatmap(self, h, w):
        """Generate Gaussian heatmap centered on image."""
        heatmap = np.zeros((h, w), dtype=np.float32)
        
        # Center of the image (person is centered in PA-100k crops)
        cx, cy = w // 2, h // 2
        
        # Create Gaussian
        sigma = min(h, w) // 8
        x = np.arange(w)
        y = np.arange(h)
        xx, yy = np.meshgrid(x, y)
        
        heatmap = np.exp(-((xx - cx)**2 + (yy - cy)**2) / (2 * sigma**2))
        
        return heatmap
    
    def __getitem__(self, idx):
        # Load image
        img_name = self.images[idx]
        img_path = os.path.join(self.root_dir, 'images', img_name)
        
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Generate targets
        hm_size = self.img_size // 4  # Stride 4 for main heatmap
        
        # Heatmap (person centered)
        heatmap = self._generate_heatmap(hm_size, hm_size)
        heatmap = torch.from_numpy(heatmap).unsqueeze(0)  # (1, H, W)
        
        # Size (normalized bbox size - full image for PA-100k)
        size = torch.tensor([0.8, 0.8])  # Person takes ~80% of cropped image
        
        # Offset (no offset needed - person is centered)
        offset = torch.tensor([0.0, 0.0])
        
        # Gender label
        if self.labels is not None:
            label = self.labels[idx]
            # Female=0, Male=1 in PA-100k
            gender = 1.0 if label[self.GENDER_MALE_IDX] == 1 else 0.0
        else:
            gender = 0.5  # Unknown
        
        gender = torch.tensor([gender], dtype=torch.float32)
        
        return {
            'image': image,
            'heatmap': heatmap,
            'size': size,
            'offset': offset,
            'gender': gender,
        }


def create_dataloaders(data_dir, batch_size=32, img_size=416, num_workers=4):
    """Create train and validation dataloaders."""
    
    # Training transforms with augmentation
    train_transform = transforms.Compose([
        transforms.Resize((img_size + 32, img_size + 32)),
        transforms.RandomCrop(img_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Validation transforms
    val_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Create datasets
    train_dataset = PA100kDataset(data_dir, split='train', img_size=img_size, transform=train_transform)
    val_dataset = PA100kDataset(data_dir, split='val', img_size=img_size, transform=val_transform)
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    return train_loader, val_loader

print("✓ Dataset classes defined")

In [ ]:
# Create dataloaders
DATA_DIR = 'data/PA-100k'
BATCH_SIZE = 32
IMG_SIZE = 416

train_loader, val_loader = create_dataloaders(
    DATA_DIR,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    num_workers=2
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# Visualize sample
import matplotlib.pyplot as plt

batch = next(iter(train_loader))

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    # Image
    img = batch['image'][i].permute(1, 2, 0).numpy()
    img = img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # Denormalize
    img = np.clip(img, 0, 1)
    
    axes[0, i].imshow(img)
    gender = 'Male' if batch['gender'][i] > 0.5 else 'Female'
    axes[0, i].set_title(f'Gender: {gender}')
    axes[0, i].axis('off')
    
    # Heatmap
    hm = batch['heatmap'][i, 0].numpy()
    axes[1, i].imshow(hm, cmap='hot')
    axes[1, i].set_title('Heatmap')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 4️⃣ Model & Training

In [ ]:
import sys
sys.path.insert(0, '.')

from mouaadnet_ultra.model import MouaadNetUltra
from mouaadnet_ultra.losses import MultiTaskLoss

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

model = MouaadNetUltra()
model = model.to(device)

print(f"Parameters: {model.count_parameters():,}")
print(f"Model size: {model.get_model_size_mb():.2f} MB")

In [ ]:
# Training configuration
EPOCHS = 50
LR = 1e-3
WEIGHT_DECAY = 1e-4

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)

# Scheduler (One-Cycle)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=LR * 10,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.3,
)

# Loss
criterion = MultiTaskLoss(
    det_weight=1.0,
    gender_weight=1.0,
    gender_pos_weight=5.0,
)

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

print("✓ Training setup complete")

In [ ]:
def train_epoch(model, loader, optimizer, scheduler, criterion, scaler, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    
    pbar = tqdm(loader, desc='Training')
    for batch in pbar:
        images = batch['image'].to(device)
        
        # Prepare targets
        targets = {
            'heatmaps': [batch['heatmap'].to(device)],
            'sizes': [batch['size'].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, images.shape[2]//4, images.shape[3]//4).to(device)],
            'offsets': [batch['offset'].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, images.shape[2]//4, images.shape[3]//4).to(device)],
            'gender_labels': batch['gender'].to(device),
        }
        
        optimizer.zero_grad()
        
        # Forward with mixed precision
        with torch.cuda.amp.autocast():
            outputs = model(images)
            
            # Reshape predictions for loss
            predictions = {
                'heatmaps': [outputs['heatmaps'][0]],  # Use first scale
                'sizes': [outputs['sizes'][0]],
                'offsets': [outputs['offsets'][0]],
                'gender': outputs['gender'],
            }
            
            losses = criterion(predictions, targets)
            loss = losses['total']
        
        # Backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': loss.item(), 'lr': scheduler.get_last_lr()[0]})
    
    return total_loss / len(loader)


@torch.no_grad()
def validate(model, loader, criterion, device):
    """Validate model."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in tqdm(loader, desc='Validating'):
        images = batch['image'].to(device)
        
        targets = {
            'heatmaps': [batch['heatmap'].to(device)],
            'sizes': [batch['size'].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, images.shape[2]//4, images.shape[3]//4).to(device)],
            'offsets': [batch['offset'].unsqueeze(-1).unsqueeze(-1).expand(-1, -1, images.shape[2]//4, images.shape[3]//4).to(device)],
            'gender_labels': batch['gender'].to(device),
        }
        
        with torch.cuda.amp.autocast():
            outputs = model(images)
            
            predictions = {
                'heatmaps': [outputs['heatmaps'][0]],
                'sizes': [outputs['sizes'][0]],
                'offsets': [outputs['offsets'][0]],
                'gender': outputs['gender'],
            }
            
            losses = criterion(predictions, targets)
            loss = losses['total']
        
        total_loss += loss.item()
        
        # Gender accuracy
        gender_pred = (torch.sigmoid(outputs['gender']) > 0.5).float()
        correct += (gender_pred == batch['gender'].to(device)).sum().item()
        total += batch['gender'].size(0)
    
    accuracy = correct / total * 100
    return total_loss / len(loader), accuracy

print("✓ Training functions defined")

In [ ]:
# Train!
best_loss = float('inf')
history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

print("="*60)
print("Starting Training")
print("="*60)

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # Train
    train_loss = train_epoch(
        model, train_loader, optimizer, scheduler, criterion, scaler, device
    )
    history['train_loss'].append(train_loss)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Save best
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
            'config': model.config,
        }, 'best_model.pt')
        print("  → Saved best model!")

print("\n" + "="*60)
print("Training Complete!")
print("="*60)

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Val')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history['val_acc'])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Gender Classification Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

## 5️⃣ Export Model

In [ ]:
# Load best model
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best model from epoch {checkpoint['epoch']}")
print(f"Best validation loss: {checkpoint['best_loss']:.4f}")

In [ ]:
# Fuse for inference
model.fuse_for_inference()
print("Model fused for inference")

In [ ]:
# Export to ONNX
dummy_input = torch.randn(1, 3, 416, 416).to(device)

torch.onnx.export(
    model,
    dummy_input,
    'mouaadnet_ultra_pa100k.onnx',
    input_names=['input'],
    output_names=['heatmap', 'size', 'offset', 'gender'],
    dynamic_axes={'input': {0: 'batch'}},
    opset_version=12,
)

print("✓ Exported to ONNX: mouaadnet_ultra_pa100k.onnx")

In [ ]:
# Download trained files
from google.colab import files

files.download('best_model.pt')
files.download('mouaadnet_ultra_pa100k.onnx')
files.download('training_history.png')

print("\n🎉 Training complete! Download your files above.")

## 📊 Final Results

In [ ]:
print("="*60)
print("🎉 MOUAADNET-ULTRA Training Complete!")
print("="*60)
print(f"\nModel: MOUAADNET-ULTRA")
print(f"Dataset: PA-100k")
print(f"Parameters: {model.count_parameters():,}")
print(f"Best Val Loss: {best_loss:.4f}")
print(f"Final Val Accuracy: {history['val_acc'][-1]:.2f}%")
print(f"\nExported files:")
print(f"  - best_model.pt (PyTorch)")
print(f"  - mouaadnet_ultra_pa100k.onnx (ONNX)")
print("="*60)